In [ ]:
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET
import os
from datetime import datetime

DATA_DIR_AUTO = "../../data/final/transcripts/auto"
DATA_DIR_MANUAL = "../../data/final/transcripts/manual"
DATA_DIR_BEFORE_REVIEW = "../../data/final/transcripts/before_review"
DATA_DIR_ASR_VERSIONS = "../../data/final/asr_outputs"
PREFERENCE_ORDER = ['whisper', 'mixed', 'scratch']

annotation2annot_duration = {}
annotation2duration = {}

def parse_xml_file(xml_file_path):

    tree = ET.parse(xml_file_path)

    root = tree.getroot()
    recording = None
    for r in root.iter('media'):
        recording = r.attrib['url']
        break
    assert recording is not None

    root = tree.getroot()
    annot_duration = -1
    for r in root.iter('annotDuration'):
        annot_duration =  r.text
        annot_duration = datetime.strptime(annot_duration, '%H:%M:%S').time()
        annot_duration = annot_duration.hour * 3600 + annot_duration.minute * 60
        break

    #TODO: this is a foo value
    duration = 0

    utterances = []
    last_end = -1
    for u in root.iter('u'):
        if u.text is not None and len(u.text) > 0:
            # if  xml_file_path and last_end > float(u.attrib['start']):
            #     print(f"Overlapping utterances in {xml_file_path}")
            #     break
            last_end = float(u.attrib['end'])
            utterances.append({
                'start': float(u.attrib['start']),
                'end': float(u.attrib['end']),
                'text': u.text,
                'speaker': u.attrib['who'].lower(),
                'source': u.attrib['source'] if 'source' in u.attrib else None
            })
            duration += float(u.attrib['end']) - float(u.attrib['start'])
    annotation2duration[os.path.basename(xml_file_path)] = max(annotation2duration.get(recording, 0), duration)
    annotation2annot_duration[os.path.basename(xml_file_path)] = max(annotation2annot_duration.get(recording, 0), annot_duration)
    # print(xml_file_path, duration, annot_duration, len(utterances))
    return recording, utterances


def transcript_version_select(transcripts):
    for p in PREFERENCE_ORDER:
        for n, t in transcripts.items():
            if p in n:
                return t
    return next(iter(transcripts.values()))

def load_transcripts(data_dir):
    recordings = {}

    for xmlfile in os.listdir(data_dir):
        if xmlfile.endswith('.xml'):
            recording, utterances = parse_xml_file(os.path.join(data_dir, xmlfile))
            annotations = recordings.get(recording, {})
            annotations[xmlfile] = utterances
            recordings[recording] = annotations
    return recordings, {annot_name: annot for _, annotations in recordings.items() for annot_name, annot in annotations.items()}

manual_recordings, manual_annotations = load_transcripts(DATA_DIR_MANUAL)
auto_recordings, auto_annotations = load_transcripts(DATA_DIR_AUTO)
before_review_recordings, before_review_annotations = load_transcripts(DATA_DIR_BEFORE_REVIEW)

auto_recordings_versions = {}
for ver_name in os.listdir(DATA_DIR_ASR_VERSIONS):
    path = os.path.join(DATA_DIR_ASR_VERSIONS, ver_name)
    if os.path.isdir(path):
        recordings, annotations = load_transcripts(path)
        auto_recordings_versions[ver_name] = annotations

In [ ]:
import matplotlib.pyplot as plt

relevant = [
    annotation2annot_duration[annot] for annot in manual_annotations.keys()
]

# Plotting the histogram
plt.hist(relevant, bins=100, edgecolor='black')

# Adding labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram')

# Displaying the histogram
plt.show()

# print annotations with duration more than 10000 minutes
for annot, duration in annotation2annot_duration.items():
    if duration > 10000:
        print(annot, duration)

In [ ]:
import matplotlib.pyplot as plt

relevant = [
    annotation2annot_duration[annot] / annotation2duration[annot] for annot in manual_annotations.keys()
]

# Plotting the histogram
plt.hist(relevant, bins=100, edgecolor='black')

# Adding labels and title
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram')

# Displaying the histogram
plt.show()

# print annotations with duration more than 10000 minutes
for annot in manual_annotations.keys():
    if annotation2annot_duration[annot] / annotation2duration[annot] > 30:
        print(annot, duration)

In [ ]:
data = []
for t in PREFERENCE_ORDER:
    relevant = [
        annotation2annot_duration[annot] / annotation2duration[annot] for annot in manual_annotations.keys() if t in annot and 10000 > annotation2annot_duration[annot] > 0
    ]
    data.append([v for v in relevant if v < 1500])

# plot three histograms in 3D plot
import numpy as np
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=20, azim=250)
for i, d in enumerate(reversed(data)):
    hist, bins = np.histogram(d, bins=100)
    xs = (bins[:-1] + bins[1:]) / 2
    ax.bar(xs, hist, zs=i, zdir='y', alpha=0.8)

fig.legend(list(reversed(PREFERENCE_ORDER)))


In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from mosestokenizer import MosesTokenizer, MosesPunctuationNormalizer
import re

PUNCT_REPLACE = re.compile(r'[\.,\?!:]+')

if not os.path.exists('alignments'):
    os.makedirs('alignments')

punct_normalizer = MosesPunctuationNormalizer('cs')
tokenizer = MosesTokenizer('cs')

excluded = set()

IS_CHAR_LEVEL = False

def normalize_text(text, char_level=False):
    text = text.replace('\n', ' ')
    text = punct_normalizer(text)
    text = text.replace('-', '')
    text = text.replace('*', '')
    text = text.replace('\'', '')
    text = re.sub(PUNCT_REPLACE, ' ', text)
    text = text.split()
    if IS_CHAR_LEVEL or char_level:
        return [t for t in ''.join(text).lower() if t.isalnum()]
    return [t.lower() for t in text if t.isalnum()]

def align_texts(text1, text2):
    text1 = normalize_text(text1)
    text2 = normalize_text(text2)
    alignments = pairwise2.align.globalxs(text1, text2, one_alignment_only=True, gap_char=['-'], open=-1, extend=-1)
    
    agree, disagree = 0, 0
    for a, b in zip(alignments[0][0], alignments[0][1]):
        if a == b:
            agree += 1
        else:
            disagree += 1
    return agree, disagree, alignments[0]

def concat_texts_by_speaker(utterances):
    speakers = set([u['speaker'] for u in utterances])
    texts = {}
    for speaker in speakers:
        texts[speaker] = ' '.join([u['text'] for u in utterances if u['speaker'] == speaker])
    return speakers, texts

## RQ1: Is manual post-editting of ASR outputs more eﬀicient than manual transcription?

### Measuring in edit distance (Levenshtein, LCS). For all transcripts.


In [ ]:
os.makedirs('alignments/rq1.1', exist_ok=True)

annot2texts = {k: concat_texts_by_speaker(v) for k, v in before_review_annotations.items()}

annot2edit_distances = {}
annot2numwords = {}
edit_distances = {}
totals = {}

for annot_name, (speakers, texts) in annot2texts.items():
    for t in PREFERENCE_ORDER:
        if t in annot_name:
            break
    src_speakers, src_texts = concat_texts_by_speaker(auto_annotations[annot_name])
    if 'scratch' in annot_name:
        src_texts = ['' for _ in src_texts]
        
    for speaker, text in texts.items():
        l = len(normalize_text(text))
        if speaker in src_speakers:
            agree, disagree, alignment = align_texts(text, src_texts[speaker] if speaker in src_speakers else '')
            if disagree / l > 0.9:
                print(f"{annot_name} {speaker} {t} {agree} {disagree}")
            with open(f'alignments/rq1.1/{annot_name}_{speaker}.txt', 'w') as f:
                f.write(format_alignment(*alignment))
        else:
            agree, disagree = 0, l
        edit_distances[t] = edit_distances.get(t, 0) + disagree
        totals[t] = totals.get(t, 0) + l
        annot2edit_distances[annot_name] = annot2edit_distances.get(annot_name, 0) + disagree
        annot2numwords[annot_name] = annot2numwords.get(annot_name, 0) + l


print('Edit distances:', edit_distances)
print('Total words:   ', totals)
print('Edits per word:', [(k, v / totals[k]) for k, v in edit_distances.items()])

columns = ['whisper', 'mixed', 'scratch']
values = [totals[c] for c in columns]
relativ = [edit_distances[c] / totals[c] for c in columns]
fig, ax = plt.subplots()
ax.bar(columns, relativ)
ax.set_ylabel('Edit distance per word')


In [ ]:
IS_CHAR_LEVEL = True
os.makedirs('alignments/rq1.1chars', exist_ok=True)

annot2texts = {k: concat_texts_by_speaker(v) for k, v in before_review_annotations.items()}

edit_distances = {}
totals = {}

for annot_name, (speakers, texts) in annot2texts.items():
    for t in PREFERENCE_ORDER:
        if t in annot_name:
            break
    src_speakers, src_texts = concat_texts_by_speaker(auto_annotations[annot_name])
    if 'scratch' in annot_name:
        src_texts = ['' for _ in src_texts]
        
    for speaker, text in texts.items():
        l = len(normalize_text(text))
        if speaker in src_speakers:
            agree, disagree, alignment = align_texts(text, src_texts[speaker] if speaker in src_speakers else '')
            if disagree / l > 0.9:
                print(f"{annot_name} {speaker} {t} {agree} {disagree}")
            with open(f'alignments/rq1.1chars/{annot_name}_{speaker}.txt', 'w') as f:
                f.write(format_alignment(*alignment))
        else:
            agree, disagree = 0, l
        edit_distances[t] = edit_distances.get(t, 0) + disagree
        totals[t] = totals.get(t, 0) + l


print('Edit distances:     ', edit_distances)
print('Total characters:   ', totals)
print('Edits per character:', [(k, v / totals[k]) for k, v in edit_distances.items()])

columns = ['whisper', 'mixed', 'scratch']
values = [totals[c] for c in columns]
relativ = [edit_distances[c] / totals[c] for c in columns]
fig, ax = plt.subplots()
ax.bar(columns, relativ)
ax.set_ylabel('Edit distance per character')
IS_CHAR_LEVEL = False


## Comparison over all recordings. Duration per word, per second (of the recording). Average edit distance (in words, chars).

In [ ]:
os.makedirs('alignments/rq1.2', exist_ok=True)

annot2texts = {k: concat_texts_by_speaker(v) for k, v in before_review_annotations.items()}

durations = {}
annotdurations = {}
edits = {}
words = {}

for annot_name, (speakers, texts) in annot2texts.items():
    for t in PREFERENCE_ORDER:
        if t in annot_name:
            break
    if -1 < annotation2annot_duration[annot_name] < 10000: #TODO: this is 
        durations[t] = durations.get(t, 0) + annotation2duration[annot_name]
        annotdurations[t] = annotdurations.get(t, 0) + annotation2annot_duration[annot_name]
        edits[t] = edits.get(t, 0) + annot2edit_distances[annot_name]
        words[t] = words.get(t, 0) + annot2numwords[annot_name]


print('Total times: ', durations)
print('Edit distances:', edits)
print('Annot to recording ratio:', [(k, annotdurations[k] / v) for k, v in durations.items()])
print('Annot time per word:', [(k, annotdurations[k] / v) for k, v in words.items()])
print('Annot duration per edit:', [(k, annotdurations[k] / v) for k, v in edits.items()])
print('Edit distance per word:', [(k, v / words[k]) for k, v in edits.items()])

### Comparison over the recordings that were both manually transcribed and manually post-edited. Total duration. Total edit distance.

In [ ]:
relevant_recordings = [k for k, v in before_review_recordings.items() if any(['scratch' in a for a in v.keys()]) and len(v) > 1]

total_words = {}
total_duration = {}
total_edits = {}
total_annot_duration = {}

for recording in relevant_recordings:
    for annot_name in auto_recordings[recording]:
        for t in PREFERENCE_ORDER:
            if t in annot_name:
                break
        if t not in annot_name:
            continue
        if -1 < annotation2annot_duration[annot_name] < 10000: #TODO: this is 
            total_words[t] = total_words.get(t, 0) + annot2numwords[annot_name]
            total_duration[t] = total_duration.get(t, 0) + annotation2duration[annot_name]
            total_edits[t] = total_edits.get(t, 0) + annot2edit_distances[annot_name]
            total_annot_duration[t] = total_annot_duration.get(t, 0) + annotation2annot_duration[annot_name]

print('Relevant recordings:', len(relevant_recordings))
print('Total words: ', total_words)
print('Total duration:', total_duration)
print('Total edits:', total_edits)
print('Total annot duration:', total_annot_duration)
print('Edits per word:', [(k, total_edits[k] / v) for k, v in total_words.items()])
print('Duration per edit:', [(k, v / total_edits[k]) for k, v in total_duration.items()])
print('Annot duration per edit:', [(k, v / total_edits[k]) for k, v in total_annot_duration.items()])

### Do the duration and edit distance correlate?
TODO

### Is there a significant difference across annotators? Some might prefer one way of transcribing over the other.


In [ ]:
def parse_annotator(annot_name):
    n = annot_name.split('from')[0]
    n = n[:-1].split('-')[-1]
    assert len(n) > 0   
    return n

total_words = {}
total_duration = {}
total_edits = {}
total_annot_duration = {}

annotators = set([parse_annotator(a) for a in auto_annotations.keys()])



for recording in manual_recordings.keys():
    for annot_name in manual_recordings[recording]:
        for t in PREFERENCE_ORDER:
            if t in annot_name:
                break
        if t not in annot_name:
            continue

        if -1 < annotation2annot_duration[annot_name] < 10000: #TODO: this is 
            annotator = parse_annotator(annot_name)
            total_words[f'{t}_{annotator}'] = total_words.get(f'{t}_{annotator}', 0) + annot2numwords[annot_name]
            total_duration[f'{t}_{annotator}'] = total_duration.get(f'{t}_{annotator}', 0) + annotation2duration[annot_name]
            total_edits[f'{t}_{annotator}'] = total_edits.get(f'{t}_{annotator}', 0) + annot2edit_distances[annot_name]
            total_annot_duration[f'{t}_{annotator}'] = total_annot_duration.get(f'{t}_{annotator}', 0) + annotation2annot_duration[annot_name]

print('Annotators:', annotators)
# print('Total words: ', total_words)
# print('Total duration:', total_duration)
# print('Total edits:', total_edits)
# print('Total annot duration:', total_annot_duration)
print('Edits per word:', sorted([(k, total_edits[k] / v) for k, v in total_words.items()], key=lambda x: x[0]))
print()
print('Annot duration per word:', sorted([(k, total_annot_duration[k] / v) for k, v in total_words.items()], key=lambda x: x[0]))
print()
# print('Duration per edit:', [(k, v / total_edits[k]) for k, v in total_duration.items()])
print('Annot duration per edit:', sorted([(k, v / total_edits[k]) for k, v in total_annot_duration.items()], key=lambda x: x[0]))


In [ ]:

columns = ['whisper', 'mixed', 'scratch']
rows = list(annotators)
values = {k: total_annot_duration[k] / v for k, v in total_words.items()}
# plot a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
sns.heatmap([[values.get(f'{c}_{r}', -1) for c in columns] for r in rows], annot=True, ax=ax, xticklabels=columns, yticklabels=rows)
ax.set_xlabel('Preference')
ax.set_ylabel('Annotator')
ax.set_title('Annotaton duration per word')


In [ ]:
columns = ['whisper', 'mixed', 'scratch']
rows = list(annotators)
values = {k: total_annot_duration[k] / v for k, v in total_duration.items()}
# plot a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
sns.heatmap([[values.get(f'{c}_{r}', -1) for c in columns] for r in rows], annot=True, ax=ax, xticklabels=columns, yticklabels=rows)
ax.set_xlabel('Preference')
ax.set_ylabel('Annotator')
ax.set_title('Annotator duration per recording duration')

### Is there a significant difference across CEFR levels?


In [ ]:
# TODO: same as above, but change the parsing of annotator name to level name


### Is there a significant difference across exercise types?

In [ ]:
# TODO

## Align AUTO to MANUAL on utterance level

Idea: concat per speaker texts -> align AUTO to MANUAL -> resegment MANUAL based on the alignment and AUTO segmentation

In [ ]:
def concat_texts_by_speaker_with_source(utterances):
    texts = {}
    for u in utterances:
        words = texts.get(u['speaker'], [])
        words.extend([
            (w, u['source']) for w in normalize_text(u['text'])
        ])
        texts[u['speaker']] = words
    return texts

def match_fn(a, b):
    a, b = set(a), set(b)
    return len(a.intersection(b)) / len(a.union(b))

relevant_annotation_names = [k for k, v in before_review_annotations.items() if 'mixed' in k]
for annot_name in relevant_annotation_names[1:]:
    before_review_texts = concat_texts_by_speaker_with_source(before_review_annotations[annot_name])
    auto_texts = concat_texts_by_speaker_with_source(auto_annotations[annot_name])

    for speaker in auto_texts.keys():
        if speaker not in before_review_texts:
            print(f"Speaker {speaker} not found in before review texts for {annot_name}")
            continue
        before_review_text = before_review_texts[speaker]
        auto_text = auto_texts[speaker]
        singletons = pairwise2.align.globalcs(auto_text, before_review_text, one_alignment_only=True, gap_char=[(None, None)], open=0, extend=0,
                                                  match_fn=lambda a, b: 1 if a[0] == b[0] else 0)
        print(f"Alignment for {annot_name} {speaker} with score {singletons[0][2]}")
        curr_auto_text, curr_br_text, curr_source = [], [], None
        for (auto_text, src), (br_text, _) in zip(singletons[0][0], singletons[0][1]):
            if curr_source == None:
                curr_source = src

            if src != None and curr_source != src:
                # if annot_name == 'A2ML_220919_01-KV-from_mixed.xml':
                #     # print(f"Current source: {curr_source}")
                #     # print(f"Auto: {' '.join(curr_auto_text)}")
                #     # print(f"BR:   {' '.join(curr_br_text)}")
                #     # print()
                curr_auto_text, curr_br_text, curr_source = [], [], src

            curr_auto_text += [auto_text] if auto_text != None else ''
            curr_br_text += [br_text] if br_text != None else ''
    #     break
    # break
            

## 

# RQ4: Is the human post-edited transcription biased towards the ASR system it was based on?
## measure WER or other ASR metrics on outputs of multiple ASR systems while varying the reference transcripts

In [ ]:
def parse_annot_type(filename):
    return filename.split('from_')[-1].split('.')[0]

def sort_by_time(utterances):
    texts = []
    for u in sorted(utterances, key=lambda x: x['start']):
        texts.append(u['text'])
    return ' '.join(texts)

wers = {}
for reference_asr, version in auto_recordings_versions.items():
    agree, disagree = dict(), dict()
    for recname, asr_transcript in version.items():
        recname = recname.replace('.xml', '.mp3')
        asr_transcript = sort_by_time(asr_transcript)
        
        if recname not in manual_recordings:
            continue
            
        for annot_name, manual_transcript in manual_recordings[recname].items():
            annot_type = parse_annot_type(annot_name)
            manual_transcript = sort_by_time(manual_transcript)


            if annot_type not in agree:
                agree[annot_type] = 0
                disagree[annot_type] = 0


            a, d, _ = align_texts(asr_transcript, manual_transcript)
            agree[annot_type] += a
            disagree[annot_type] += d
    print(f"ASR version {reference_asr}:")
    for annot_type in agree.keys():
        print(f"  {annot_type}: {agree[annot_type]} / {disagree[annot_type]}; WER: {disagree[annot_type] / (agree[annot_type] + disagree[annot_type])}")
        wers[f'{reference_asr}_{annot_type}'] = disagree[annot_type] / (agree[annot_type] + disagree[annot_type])


In [ ]:
# plot a heatmap
import seaborn as sns
import matplotlib.pyplot as plt

print(wers)
fig, ax = plt.subplots()
columns = set(c.split('_')[1] for c in wers.keys())
rows = set(c.split('_')[0] for c in wers.keys())
sns.heatmap([[wers.get(f'{r}_{c}', -1) for c in columns] for r in rows], annot=True, ax=ax, xticklabels=columns, yticklabels=rows)
ax.set_xlabel('Preference')
ax.set_ylabel('ASR version')
ax.set_title('Word error rate')

# Auto to Manual Utterances alignment

In [ ]:
from Bio import pairwise2
def parse_annot_type(filename):
    return filename.split('from_')[-1].split('.')[0]

def sort_by_time(utterances):
    return sorted(utterances, key=lambda x: x['start'])

def concat_adjacent_speaker_utterances(utterances):
    if len(utterances) == 0:
        return []
    new_utterances = [utterances[0].copy()]
    new_utterances[-1]['original_utterances'] = [0]

    for idx, u in enumerate(utterances[1:]):
        if new_utterances[-1]['speaker'] == u['speaker']:
            new_utterances[-1]['text'] += ' ' + u['text']
            new_utterances[-1]['end'] = u['end']
            new_utterances[-1]['original_utterances'].append(idx + 1)
        else:
            new_utterances.append(u.copy())
            new_utterances[-1]['original_utterances'] = [idx + 1]
    return new_utterances

def time_overlap(a, b) -> float:
    """
    Return the overlap of two time intervals
    if the intervals do not overlap, return the distance between them
    """
    if a['start'] > b['end'] or b['start'] > a['end']:
        return min(a['end'], b['end']) - max(a['start'], b['start'])
    return min(a['end'], b['end']) - max(a['start'], b['start'])

def text_score(a, b):
    a, b = normalize_text(a['text'], char_level=True), normalize_text(b['text'], char_level=True)
    if len(a) == 0 or len(b) == 0:
        return 0
    alignment = pairwise2.align.globalms(a, b, one_alignment_only=True, gap_char=['-'], open=0, extend=0, match=1, mismatch=0)
    return alignment[0].score

def match_fn_with_threshold(a, b, threshold=0.1):
    overlap = time_overlap(a, b)
    if overlap < -threshold:
        return float('-inf')
    l = b['end'] - b['start']
    return (overlap / l) * text_score(a, b) / max(1, len(normalize_text(b['text'], char_level=True))) 

def find_best_alignment_for_auto(alignment, idx):
    candidates = []
    for i in range(idx - 1, -1, -1):
        if alignment[0][i] != None:
            candidates.append(alignment[0][i])
        '''
        we need at least 2 utterances
        since the manual transcript is concatenated
        we know the 2 utterances have different speakers
        '''
        if len(candidates) > 1:  
            break

    for i in range(idx + 1, len(alignment[0])):
        if alignment[0][i] != None:
            candidates.append(alignment[0][i])
        if len(candidates) > 3: # two utterances from the right side
            break

    match_fn = lambda a, b: match_fn_with_threshold(a, b, threshold=0.5) # be less strict
    scores = [match_fn(c, alignment[1][idx]) for c in candidates]
    
    if len(scores) == 0 or max(scores) <= 0:
        return None
    return candidates[scores.index(max(scores))]

def align_utterances(manual_utterances, asr_utterances):
    manual_utterances = sort_by_time(manual_utterances)
    manual_utterances = concat_adjacent_speaker_utterances(manual_utterances)

    asr_utterances = sort_by_time(asr_utterances)
    for idx, u in enumerate(asr_utterances):
        asr_utterances[idx]['original_utterances'] = [idx]

    match_fn = lambda a, b: match_fn_with_threshold(a, b, threshold=0.0) # be more strict
    alignment = pairwise2.align.globalcx(manual_utterances, asr_utterances, one_alignment_only=True, gap_char=[None], match_fn=match_fn)
    return alignment
    

wers = {}
with open('alignments.txt', 'w') as f:
    for reference_asr, version in auto_recordings_versions.items():
        agree, disagree = dict(), dict()
        for recname, asr_transcript in version.items():
            recname = recname.replace('.xml', '.mp3')
            
            if recname not in manual_recordings:
                continue
                
            f.write(f"ASR version {reference_asr} for {recname}:\n")
            for annot_name, manual_transcript in manual_recordings[recname].items():
                annot_type = parse_annot_type(annot_name)

                alignment = align_utterances(manual_transcript,asr_transcript, )
                for idx, (a, b) in enumerate(zip(alignment[0][0], alignment[0][1])):
                    if b is not None:
                        al = []
                        if a is None:
                            a = find_best_alignment_for_auto(alignment[0], idx)
                        if a is not None:
                            al.extend(a['original_utterances'])
                        # print(b['original_utterances'])
                        f.write(f"{b['original_utterances']} -> {al}\n")
        #         break
        #     break
        # break


## Notes
 - Using WhisperX segmentation biases the number of utterances (e.g., 07-npi-test-B1.mp3 with 161 and 273)

## Basic Stats

In [ ]:
# TODO: how to compute the total duration?
total_duration = sum(list(recording.values())[0][-1]['end'] - list(recording.values())[0][0]['start'] for recording in recordings.values())
print(f"Total recordings:  {len(recordings)}")
print(f"Total duration:    {total_duration:.02f} seconds ({total_duration/60:.02f} minutes)")

# TODO: should we take the max or select some specific version?
total_utterances = sum(max(len(utterances) for utterances in transcripts.values()) for transcripts in recordings.values())
print(f"Total utterances:  {total_utterances}")

total_words = sum(
    max(
        sum(
            len(utterance['text'].split())
            for utterance in transcript
        )
        for transcript in recording.values()
    )
 for recording in recordings.values())
print(f"Total words:       {total_words}")


num_versions_histogram = {}
for recording, transcripts in recordings.items():
    num_versions = len(transcripts)
    num_versions_histogram[num_versions] = num_versions_histogram.get(num_versions, 0) + 1
print(f"Version histogram: {num_versions_histogram}")

num_speakers_histogram = {}
for recording, transcripts in recordings.items():
    num_speakers = max(len(set([u['speaker'] for u in utterances])) for utterances in transcripts.values())
    num_speakers_histogram[num_speakers] = num_speakers_histogram.get(num_speakers, 0) + 1
print(f"Speaker histogram: {num_speakers_histogram}")


## Students Stats

In [ ]:
total_student_duration = sum(
    max(
        sum(
            utterance['end'] - utterance['start']
            for utterance in transcript
            if utterance['speaker'] != 't'
        )
        for transcript in recording.values()
    )
 for recording in recordings.values())
print(f"Total student duration:    {total_student_duration:.02f} seconds ({total_student_duration/60:.02f} minutes); {total_student_duration/total_duration*100:.02f}% of total duration")

total_student_utterances = sum(
    max(
        sum(
            utterance['speaker'] != 't'
            for utterance in transcript
        )
        for transcript in recording.values()
    )
 for recording in recordings.values())
print(f"Total student utterances:  {total_student_utterances} ({total_student_utterances/total_utterances*100:.02f}% of total utterances)")

total_student_words = sum(
    max(
        sum(
            len(utterance['text'].split())
            for utterance in transcript
            if utterance['speaker'] != 't'
        )
        for transcript in recording.values()
    )
 for recording in recordings.values())
print(f"Total student words:       {total_student_words} ({total_student_words/total_words*100:.02f}% of total words)")

## Level Stats

In [ ]:
for level in ['A1', 'A2', 'B1', 'B2', 'C1']:
    relevant = [transcript_version_select(r) for n, r in recordings.items() if level in n]
    
    duration = sum(transcript[-1]['end'] - transcript[0]['start'] for transcript in relevant)
    print(f'{level}: {len(relevant)} recordings; {duration:.02f} seconds ({duration/60:.02f} minutes)')

    

## Inter-annotator Agreement

In [ ]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from mosestokenizer import MosesTokenizer, MosesPunctuationNormalizer

if not os.path.exists('alignments'):
    os.makedirs('alignments')

punct_normalizer = MosesPunctuationNormalizer('cs')
tokenizer = MosesTokenizer('cs')

def normalize_text(text):
    text = text.lower().replace('\n', " ")
    text = punct_normalizer(text)
    text = tokenizer(text)
    return [t for t in text if t.isalnum()]

def align_texts_w(text1, text2):
    text1 = normalize_text(text1)
    text2 = normalize_text(text2)
    alignments = pairwise2.align.globalxs(text1, text2, one_alignment_only=True, gap_char=['-'], open=-1, extend=-1)
    
    agree, disagree = 0, 0
    for a, b in zip(alignments[0][0], alignments[0][1]):
        if a == b:
            agree += 1
        else:
            disagree += 1
    return agree, disagree, alignments[0]

def align_texts(text1, text2):
    text1 = list("".join(normalize_text(text1)))
    text2 = list("".join(normalize_text(text2)))
    alignments = pairwise2.align.globalxs(text1, text2, one_alignment_only=True, open=-1, extend=-1, gap_char=['-'])
    
    agree, disagree = 0, 0
    for a, b in zip(alignments[0][0], alignments[0][1]):
        if a == b:
            agree += 1
        else:
            disagree += 1
    return agree, disagree, alignments[0]

def concat_texts_by_speaker(utterances):
    speakers = set([u['speaker'] for u in utterances])
    texts = {}
    for speaker in speakers:
        texts[speaker] = ' '.join([u['text'] for u in utterances if u['speaker'] == speaker])
    return speakers, texts

relevant = {n: r for n, r in before_review_recordings.items() if len(r) > 1}

def get_type(name):
    if 'whisper' in name:
        return 'whisper'
    if 'mixed' in name:
        return 'mixed'
    if 'scratch' in name:
        return 'scratch'
    return 'unknown'

agreements = {}
disagreements = {}

for name, transcripts in relevant.items():
    print(f"Recording {name}")
    for idx, (n1, t1) in enumerate(list(transcripts.items())):
        speakers, t1 = concat_texts_by_speaker(t1)
        for n2, t2 in list(transcripts.items())[idx+1:]:
            speakers, t2 = concat_texts_by_speaker(t2)
            for speaker in speakers:
                if speaker not in t1 or speaker not in t2:
                    continue
                agree, disagree, alignment = align_texts(t1[speaker], t2[speaker])
                
                speaker = 'teacher' if speaker == 't' else 'student ' + speaker
                print(f"  {n1} vs {n2} ({speaker}):\t {agree/(agree+disagree)*100:.02f}% agreement\t {agree} agree, {disagree} disagree")
                if agree / (agree + disagree) < 0.25:
                    continue
                agreements[get_type(n1), get_type(n2)] = agreements.get((get_type(n1), get_type(n2)), 0) + agree
                disagreements[get_type(n1), get_type(n2)] = disagreements.get((get_type(n1), get_type(n2)), 0) + disagree
                
                agreements[get_type(n2), get_type(n1)] = agreements[get_type(n1), get_type(n2)]
                disagreements[get_type(n2), get_type(n1)] = disagreements[get_type(n1), get_type(n2)]

                with open(f'alignments/{name}_{n1}_{n2}_{speaker}.txt', 'w') as f:
                    f.write(format_alignment(*alignment))
                    f.write(f"\n{agree/(agree+disagree)*100:.02f}% agreement\t {agree} agree, {disagree} disagree\n")



print(agreements)
print(disagreements)
print({k: v / (disagreements[k] + v) for k, v in agreements.items()})

# plot confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
sns.heatmap([[agreements.get((r, c), 0) / (disagreements.get((r, c), 0) + agreements.get((r, c), 1)) for c in ['whisper', 'mixed', 'scratch']] for r in ['whisper', 'mixed', 'scratch']], annot=True, ax=ax, xticklabels=['whisper', 'mixed', 'scratch'], yticklabels=['whisper', 'mixed', 'scratch'])
ax.set_xlabel('Reference')
ax.set_ylabel('Hypothesis')
